In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 2
setup_one_gpu(GPU)

Picking GPU 2


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph
from general_tools.in_out.basics import create_dir, files_in_subdirs

from geo_tool import Point_Cloud

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry, decoder_with_fc_only
from tf_lab.point_clouds.in_out import load_point_clouds_from_filenames, PointCloudDataSet

from tf_lab.in_out.basics import Data_Splitter, read_saved_epochs          
from tf_lab.data_sets.shape_net import snc_category_to_synth_id, load_all_point_clouds_under_folder
from tf_lab.data_sets.shape_net import pc_loader as snc_loader

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
n_pc_points = 2048
do_training = True
first_time_running = True
load_pretrained_model = False
load_epoch = None
seed = 42

In [5]:
max_training_epochs = 1000
loss = 'chamfer'
bneck_size = 512

class_name = raw_input('Give me the class type: ').lower()
syn_id = snc_category_to_synth_id()[class_name]

experiment_id = raw_input('Give me the experiment ID:').lower()

experiment_name = '_'.join([class_name, experiment_id, str(n_pc_points), 
                           'pts_bneck', str(bneck_size), loss
                           ])

top_data_dir = '/orions4-zfs/projects/optas/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/iclr/nn_models/testing_ae_settings', experiment_name)
create_dir(train_dir)

Give me the class type: chair
Give me the experiment ID:conv_arch_0_with_dropout


'/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/testing_ae_settings/chair_conv_arch_0_with_dropout_2048_pts_bneck_512_chamfer'

In [6]:
train_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits', syn_id, '85_5_10/train.txt')
test_split = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Splits/single_class_splits', syn_id, '85_5_10/test.txt')

top_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_points))
splitter = Data_Splitter(top_pclouds_path, data_file_ending='.ply')

In [7]:
tr_files = splitter.load_splits(train_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(tr_files, n_threads=20, loader=snc_loader, verbose=True)
train_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

test_files = splitter.load_splits(test_split)
pclouds, model_ids, syn_ids = load_point_clouds_from_filenames(test_files, n_threads=20, loader=snc_loader, verbose=True)
test_data = PointCloudDataSet(pclouds, labels=syn_ids + '_' + model_ids)

5761 pclouds were loaded. They belong in 1 shape-classes.
679 pclouds were loaded. They belong in 1 shape-classes.


In [34]:
encoder_args = { 'n_filters': [128, 128, 256, bneck_size],
                 'filter_sizes': [40, 20, 10, 10],
                 'strides': [1, 1, 2, 2],
                 'b_norm': False,
                 'dropout_prob': [0.5, 0, 0.5, 0]
                }

decoder_args = {'layer_sizes': [1024, 2048, n_pc_points*3], 
                'b_norm': False,
                'dropout_prob': [0, 0.5, 0]}

In [35]:
if load_pretrained_model:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    print conf
    if conf.train_dir != train_dir:
        conf.train_dir = train_dir
    conf.save(osp.join(train_dir, 'configuration'))
else:
    conf = Conf(
                n_input = [n_pc_points, 3],
                loss = loss,
                training_epochs = 1000,
                batch_size = 50,
                denoising = False,
                learning_rate = 0.0005,
                train_dir = train_dir,              
                loss_display_step = 1,                
                saver_step = 20,
                z_rotate = False,
                encoder = encoder_with_convs_and_symmetry,
                decoder = decoder_with_fc_only,
                encoder_args = encoder_args,
                decoder_args = decoder_args
               )    
    conf.experiment_name = experiment_name
    conf.held_out_step = 5
    conf.save(osp.join(conf.train_dir, 'configuration'))

In [36]:
reset_tf_graph()
ae = PointNetAutoEncoder(conf.experiment_name, conf)

In [37]:
if load_pretrained_model:
    if load_epoch is None: # load last saved.
        saved_epochs = read_saved_epochs(conf.train_dir)
        load_epoch = saved_epochs[-1]        
    ae.restore_model(conf.train_dir, load_epoch, verbose=True)

In [38]:
if do_training:
    buf_size = 1 # flush each line
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
    train_stats = ae.train(train_data, conf, log_file=fout, held_out_data=test_data)
    fout.close()

('Epoch:', '0001', 'training time (minutes)=', '0.1889', 'loss=', '0.005150118')
INFO:tensorflow:/orions4-zfs/projects/optas/DATA/OUT/iclr/nn_models/testing_ae_settings/chair_conv_arch_0_with_dropout_2048_pts_bneck_512_chamfer/models.ckpt-1 is not in all_model_checkpoint_paths. Manually adding it.
('Epoch:', '0002', 'training time (minutes)=', '0.1887', 'loss=', '0.002970552')
('Epoch:', '0003', 'training time (minutes)=', '0.1903', 'loss=', '0.002399511')
('Epoch:', '0004', 'training time (minutes)=', '0.1917', 'loss=', '0.002183501')
('Epoch:', '0005', 'training time (minutes)=', '0.1925', 'loss=', '0.002038921')
('Held Out Data :', 'forward time (minutes)=', '0.0057', 'loss=', '0.011872458')
('Epoch:', '0006', 'training time (minutes)=', '0.1931', 'loss=', '0.001939399')
('Epoch:', '0007', 'training time (minutes)=', '0.1954', 'loss=', '0.001846412')
('Epoch:', '0008', 'training time (minutes)=', '0.1969', 'loss=', '0.001772740')
('Epoch:', '0009', 'training time (minutes)=', '0.206